In [1]:
from miso.hist_features import get_features
from miso.utils import *
from miso import Miso
from PIL import Image
import pandas as pd
import numpy as np
import scanpy as sc
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
Image.MAX_IMAGE_PIXELS = None
Image.MAX_IMAGE_PIXELS = None
import torch
import random

seed=100
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

if torch.cuda.is_available():
    device = 'cuda:0'
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("CUDA is available. GPU:", torch.cuda.get_device_name(0))
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")


CUDA is available. GPU: NVIDIA GeForce RTX 3090


In [3]:
#Load data and perform necessary preprocessing

path="../datasets/10x_human_lymph_node_D1/"

rna = sc.read_h5ad(path+"adata_RNA.h5ad")
rna = preprocess(rna,modality='rna')

protein = sc.read_h5ad(path+"adata_ADT.h5ad")
protein = preprocess(protein,modality='protein')
#if using a subset of modality-specific terms, the "ind_views" parameter should be a list with values entries to the indices of the modalities to be included, e.g.,  ind_views=[0,2] if including RNA and image features
#if using a subset of interaction terms, the "combs" parameter should be a list of tuples with entries to the indices of the modalities for each interaction, e.g. combs = [(0,1),(0,2)] if including the RNA-protein and RNA-image interaction terms
model = Miso([rna,protein],ind_views='all',combs='all',sparse=False,device=device)
model.train()

clusters = model.cluster(n_clusters=7)

adata=sc.read_h5ad(path+"adata_RNA.h5ad")
# adata.obs["cluster"]=clusters
# adata.obs["cluster"]=adata.obs["cluster"].astype("str")
adata.obsm["X_miso"]=model.emb
adata.write_h5ad(path.replace("datasets","results")+"adata_miso.h5ad")

Training network for modality 1:   0%|          | 0/1000 [00:00<?, ?it/s]

Training network for modality 2:   0%|          | 0/1000 [00:00<?, ?it/s]

/home/ws6tg/anaconda3/envs/scvi/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
#Load data and perform necessary preprocessing

path="../datasets/simulated_data/"

rna = sc.read_h5ad(path+"adata_RNA.h5ad")
rna = preprocess(rna,modality='rna')

protein = sc.read_h5ad(path+"adata_ADT.h5ad")
protein = preprocess(protein,modality='protein')

atac = sc.read_h5ad(path+"adata_ATAC.h5ad")
atac = preprocess(atac,modality='atac')
#if using a subset of modality-specific terms, the "ind_views" parameter should be a list with values entries to the indices of the modalities to be included, e.g.,  ind_views=[0,2] if including RNA and image features
#if using a subset of interaction terms, the "combs" parameter should be a list of tuples with entries to the indices of the modalities for each interaction, e.g. combs = [(0,1),(0,2)] if including the RNA-protein and RNA-image interaction terms
model = Miso([rna,protein,atac],ind_views='all',combs='all',sparse=False,device=device)
model.train()

clusters = model.cluster(n_clusters=5)

adata=sc.read_h5ad(path+"adata_RNA.h5ad")

# adata.obs["cluster"]=adata.obs["cluster"].astype("str")
adata.obsm["X_miso"]=model.emb
adata.write_h5ad(path.replace("datasets","results")+"adata_miso.h5ad")

Training network for modality 1:   0%|          | 0/1000 [00:00<?, ?it/s]

Training network for modality 2:   0%|          | 0/1000 [00:00<?, ?it/s]

Training network for modality 3:   0%|          | 0/1000 [00:00<?, ?it/s]